<a href="https://colab.research.google.com/github/alex2844/youtube-music/blob/master/ymusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# @title Settings

import sys, os, json
from google.colab import drive
from ipywidgets import Layout, Label, Box, Dropdown, VBox, HBox, Output, Text, Button, RadioButtons
from IPython.display import display, clear_output
if not 'jupyter_ui_poll' in sys.modules:
    ! pip install jupyter_ui_poll
from jupyter_ui_poll import ui_events

os.environ['version'] = 'master'
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
if not os.path.exists('/content/drive/My Drive/ymusic'):
    os.mkdir('/content/drive/My Drive/ymusic')
os.chdir('/content/drive/My Drive/ymusic')
clear_output()

config = {}
filepath = os.path.expanduser("config.json")
if os.path.exists(filepath):
    with open(filepath) as f:
        config = json.load(f)
if not 'headers' in config:
    config['headers'] = {}
if not 'oauth' in config:
    config['oauth'] = {}

label_layout = Layout(width='150px')
input_layout = Layout(width='400px')
container_layout = Layout(width='580px', display='flex', flex_flow='column')
section_layout = Layout(width='580px', display='flex', flex_flow='column', border='1px solid lightgray', padding='10px', margin='0px 0px 10px 0px')

headers = Box(children=[
    HBox([
        Label('authorization:', layout=label_layout),
        Text(placeholder='authorization', value=config['headers'].get('authorization'), layout=input_layout
    )]),
    HBox([
        Label('cookie:', layout=label_layout),
        Text(placeholder='cookie', value=config['headers'].get('cookie'), layout=input_layout)
    ]),
    HBox([
        Label('user-agent:', layout=label_layout),
        Text(placeholder='user-agent', value=config['headers'].get('user-agent'), layout=input_layout)
    ]),
    HBox([
        Label('x-origin:', layout=label_layout),
        Text(placeholder='x-origin', value=config['headers'].get('x-origin'), layout=input_layout)
    ]),
    HBox([
        Label('x-goog-authuser:', layout=label_layout),
        Text(placeholder='x-goog-authuser', value=config['headers'].get('x-goog-authuser'), layout=input_layout)
    ])
], layout=section_layout)

oauth = Box(children=[
    HBox([
        Label('client_id:', layout=label_layout),
        Text(placeholder='client_id', value=config['oauth'].get('client_id'), layout=input_layout)
    ]),
    HBox([
        Label('client_secret:', layout=label_layout),
        Text(placeholder='client_secret', value=config['oauth'].get('client_secret'), layout=input_layout)
    ])
], layout=section_layout)

output_type = Output()
output_download = Output()

def on_type_change(change):
    with output_type:
        clear_output(wait=True)
        if change['new'] == 'headers':
            display(headers)
        elif change['new'] == 'oauth':
            display(oauth)

sel_type = RadioButtons(
    options=['headers', 'oauth'],
    value=config.get('auth', 'headers'),
    description='Auth:',
    layout=Layout(width='200px')
)
sel_type.observe(on_type_change, names='value')
on_type_change({'new': sel_type.value})

start = False
def on_start_click(click):
    global start
    start = True
btn_start = Button(
    description="Start",
    layout=Layout(width='150px')
)
btn_start.on_click(on_start_click)

sel_save = RadioButtons(
    options=['mp3', 'zip'],
    description='Save:',
    layout=Layout(width='200px')
)

input_download = Text(
    placeholder='id or url',
    layout=Layout(width='250px')
)

def on_download_change(change):
    with output_download:
        clear_output(wait=True)
        if change['new'] == 'liked':
            display(VBox([
                HBox([sel_download]),
                HBox([sel_save, btn_start])
            ]))
        else:
            display(VBox([
                HBox([sel_download, input_download]),
                HBox([sel_save, btn_start])
            ]))
sel_download = Dropdown(
    options=['liked', 'playlist', 'track'],
    description='Download:',
    layout=Layout(width='200px')
)
sel_download.observe(on_download_change, names='value')
on_download_change({'new': sel_download.value})

main_container = VBox([
    HBox([sel_type, output_type]),
    HBox([output_download])
], layout=Layout(width = 'auto', padding = '10px'))

display(main_container)

with ui_events() as poll:
    while not start:
        poll(10)

clear_output()
config['auth'] = sel_type.value
if sel_type.value == 'headers':
    for el in headers.children:
        config['headers'][el.children[1].placeholder] = el.children[1].value
elif sel_type.value == 'oauth':
    for el in oauth.children:
        config['oauth'][el.children[1].placeholder] = el.children[1].value
with open(filepath, 'w') as f:
    json.dump(config, f, indent=4)
os.environ['args'] = ' '.join([option.strip() for option in ['--'+sel_download.value, input_download.value]])

In [6]:
# @title Requirements
import requests, os
from importlib.metadata import version as get_version, PackageNotFoundError
from packaging import version
from io import StringIO
from IPython.display import clear_output

def get_installed_version(package_name):
    try:
        return get_version(package_name)
    except PackageNotFoundError:
        return None

def install_package(requirement):
    ! pip install "$requirement"

def upgrade_package(requirement):
    ! pip install --upgrade "$requirement"

def get_requirements(file):
    if file.startswith('http://') or file.startswith('https://'):
        response = requests.get(file)
        response.raise_for_status()
        return StringIO(response.text)
    else:
        return open(file, 'r')

def main(file='requirements.txt'):
    with get_requirements(file) as f:
        for line in f:
            requirement = line.strip()
            if not requirement or requirement.startswith('#'):
                continue
            parts = requirement.split('==')
            package_name = parts[0]
            required_version = parts[1] if len(parts) > 1 else None
            installed_version = get_installed_version(package_name)
            if installed_version is None:
                print(f"Устанавливаем {requirement}")
                install_package(requirement)
            elif required_version:
                if version.parse(installed_version) < version.parse(required_version):
                    print(f"Обновляем {package_name} с {installed_version} до {required_version}")
                    upgrade_package(requirement)
                else:
                    print(f"{package_name} {installed_version} уже установлен (требуется {required_version})")
            else:
                print(f"{package_name} {installed_version} уже установлен")

if __name__ == '__main__':
    main('https://raw.githubusercontent.com/alex2844/youtube-music/'+os.environ['version']+'/requirements.txt')
    clear_output()

In [ ]:
# @title Download
import os, zipfile
if os.path.exists('ymusic.zip'):
    with zipfile.ZipFile('ymusic.zip', 'r') as zipf:
        zipf.extractall()
! python <(wget -qO- "https://raw.githubusercontent.com/alex2844/youtube-music/$version/ymusic.py") ${args[@]}

In [8]:
# @title ZIP
import os, zipfile
from glob import glob
if sel_save.value == 'zip':
    with zipfile.ZipFile('ymusic.zip', 'w') as zipf:
        for mp3 in glob("*.mp3"):
            zipf.write(mp3);
            os.remove(mp3)